# Statistical testing in binary classification

## Neccessary stuff
1. Train two or more models.
1. Evaluate trained models on one or more test datasets (it's a little bit tricky when you want to make several datasets from one).
1. Ask a question you want to answer about this models.
1. Choose statistical test equivalent to your question.
1. Measure neccessary things for this test.
1. Assume some significance level, e.g. $\alpha = 0.5$. 
1. Calculate p-value for the test.

In [1]:
import os
import shutil
from random import shuffle
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-06-03 22:03:20.965012: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 22:03:21.909375: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Two models, one data set, what can I do?

### Data

In [3]:
import os

# The path to store trained models
models_dir = './models/'
if not os.path.exists(models_dir):
    os.mkdir(models_dir)

# The path to the directory where the original dataset was uncompressed
original_dataset_dir = './Dogs-vs-Cats-1'
original_cat_dir = './Dogs-vs-Cats-1/Cat'
original_dog_dir = './Dogs-vs-Cats-1/Dog'

# The directory where we will store our smaller dataset
base_dir = './Dogs-vs-Cats-1/working'
if not os.path.exists(base_dir):
    os.mkdir(base_dir)

# Directories for our training, validation and test splits
train_dir = os.path.join(base_dir, 'train')
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

# Directory with training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
if not os.path.exists(train_cats_dir):
    os.mkdir(train_cats_dir)

# Directory with training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
if not os.path.exists(train_dogs_dir):
    os.mkdir(train_dogs_dir)

# Directory with validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
if not os.path.exists(validation_cats_dir):
    os.mkdir(validation_cats_dir)
    
# Directory with validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
if not os.path.exists(validation_dogs_dir):
    os.mkdir(validation_dogs_dir)

# Directory with test cat pictures
test_cats_dir = os.path.join(test_dir, 'cats')
if not os.path.exists(test_cats_dir):
    os.mkdir(test_cats_dir)

# Directory with test dog pictures
test_dogs_dir = os.path.join(test_dir, 'dogs')
if not os.path.exists(test_dogs_dir):
    os.mkdir(test_dogs_dir)

In [36]:
# Copy first 1000 cat images to train_cats_dir
fnames = ['{}.jpg'.format(i) for i in range(1001)]
for fname in fnames:
    if fname == '666.jpg':
        continue
    src = os.path.join(original_cat_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# Copy next 500 cat images to validation_cats_dir
fnames = ['{}.jpg'.format(i) for i in range(1001, 1501)]
for fname in fnames:
    src = os.path.join(original_cat_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 cat images to test_cats_dir
fnames = ['{}.jpg'.format(i) for i in range(1501, 2001)]
for fname in fnames:
    src = os.path.join(original_cat_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy first 1000 dog images to train_dogs_dir
fnames = ['{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dog_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to validation_dogs_dir
fnames = ['{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dog_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to test_dogs_dir
fnames = ['{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dog_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [20]:
img_rows = 150
img_cols = 150

# data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size, 
        class_mode='binary')

val_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size, 
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


Found 1000 images belonging to 2 classes.


### Models

In [30]:
# First model
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_rows, img_cols, 3)))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(128, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(256, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(256, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Flatten())
model1.add(Dropout(0.5))
model1.add(Dense(512, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

/home/tosiek/Documents/VI_semestr/GiGSN/statistical-significance-ml/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [31]:
model1.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=3e-4),
              metrics=['acc'])

model1.fit(
      train_generator,
      batch_size=batch_size,
      epochs=30,
      validation_data=val_generator)

Epoch 1/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 32s 473ms/step - acc: 0.5028 - loss: 0.6946 - val_acc: 0.5010 - val_loss: 0.6925
Epoch 2/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 30s 465ms/step - acc: 0.5406 - loss: 0.6914 - val_acc: 0.5220 - val_loss: 0.6869
Epoch 3/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 30s 464ms/step - acc: 0.5575 - loss: 0.6885 - val_acc: 0.5690 - val_loss: 0.6763
Epoch 4/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 30s 455ms/step - acc: 0.5764 - loss: 0.6805 - val_acc: 0.6270 - val_loss: 0.6659
Epoch 5/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 30s 461ms/step - acc: 0.5898 - loss: 0.6742 - val_acc: 0.5860 - val_loss: 0.6650
Epoch 6/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 29s 451ms/step - acc: 0.6102 - loss: 0.6653 - val_acc: 0.6640 - val_loss: 0.6384
Epoch 7/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 30s 466ms/step - acc: 0.6188 - loss: 0.6545 - val_acc: 0.5980 - val_loss: 0.6553
Epoch 8/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 30s 463ms/step - acc: 0.6367 - loss: 0.6392 - val_acc: 0.6880 - val_loss: 0.6157
Epoch 9/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 29s 438ms/

In [32]:
model1.save(models_dir+"binary_model1.h5")

### Want to know if one model is better than other?
Perform McNemar's test!